In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import threading
import time

class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        
    def error(self, reqId, errorCode, errorString, *args):
        print(f"Error {errorCode}: {errorString}")

def main():
    app = TestApp()
    app.connect("127.0.0.1", 4002, 0)  # IB Gateway paper trading port
    
    # Start a thread for the connection
    thread = threading.Thread(target=app.run)
    thread.start()
    
    time.sleep(3)  # Give it time to connect
    
    if app.isConnected():
        print("Successfully connected to IB Gateway!")
    else:
        print("Failed to connect to IB Gateway")
    
    app.disconnect()

if __name__ == "__main__":
    main()

In [ ]:
#!/usr/bin/env python3
"""
IBKR Position Analyzer

This script connects to your Interactive Brokers account and retrieves information
about all your open positions with a position size not equal to 0.
It displays the positions in a pandas DataFrame.

Requirements:
- ibapi package (pip install ibapi)
- pandas (pip install pandas)
- Interactive Brokers IB Gateway running
"""

import time
import threading
import pandas as pd
from datetime import datetime
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import TickerId, BarData
import random


class IBKRApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        
        # Store positions and market data
        self.positions = {}  # Dictionary to store position data
        self.options_data = {}  # Dictionary to store option market data
        self.vertical_spreads = []  # List to store identified vertical spreads
        self.req_id_to_option = {}  # Maps request IDs to option contracts
        self.next_req_id = 1
        self.position_end_received = False
        self.account_update_end_received = False
        self.connection_problem = False
        
    def nextOrderId(self):
        """Get the next request ID and increment the counter"""
        id = self.next_req_id
        self.next_req_id += 1
        return id
    
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        """Handle error messages from IB Gateway"""
        # Informational messages
        if errorCode in [2104, 2106, 2158]:
            # These are connection confirmations, not errors
            return
        
        # Actual errors
        if errorCode not in [2104, 2106, 2158]:
            print(f"Error {errorCode}: {errorString}")
            if errorCode == 504:
                self.connection_problem = True
    
    def isConnectedAndReady(self):
        """Check if we're connected and no connection problems reported"""
        return self.isConnected() and not self.connection_problem
    
    def connectTWS(self, host='127.0.0.1', port=4002, clientId=1):
        """Connect to IB Gateway"""
        self.connect(host, port, clientId)
        
        # Start a thread to process messages
        thread = threading.Thread(target=self.run)
        thread.daemon = True
        thread.start()
        
        # Allow time for connection to establish
        time.sleep(2)
    
    def position(self, account, contract, position, avgCost):
        """Handle position data"""
        print(f"Position received: {contract.symbol}, {position} @ {avgCost}")
        
        if position != 0:  # Consider all non-zero positions
            # Store basic position info
            # For options, include strike, right, and expiry
            if contract.secType == "OPT":
                key = (contract.symbol, contract.secType, contract.strike, contract.right, contract.lastTradeDateOrContractMonth)
            else:
                # For stocks and other instruments
                key = (contract.symbol, contract.secType, None, None, None)
                
            self.positions[key] = {
                'contract': contract,
                'position': position,
                'avgCost': avgCost,
                'account': account
            }
            
            # Request market data for this instrument
            req_id = self.nextOrderId()
            self.req_id_to_option[req_id] = key
            self.reqMktData(req_id, contract, "", False, False, [])
    
    def positionEnd(self):
        """Called when all positions have been reported"""
        print("Position end received")
        self.position_end_received = True
        self.find_vertical_spreads()
        
    def accountUpdateMulti(self, reqId, account, modelCode, key, value, currency):
        """Handle account update data"""
        print(f"Account Update: {key}={value} {currency}")
    
    def accountUpdateMultiEnd(self, reqId):
        """Called when all account updates have been reported"""
        self.account_update_end_received = True
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Handle price updates for options"""
        if reqId in self.req_id_to_option and tickType == 4:  # 4 is last price
            option_key = self.req_id_to_option[reqId]
            if option_key not in self.options_data:
                self.options_data[option_key] = {}
            self.options_data[option_key]['price'] = price
    
    def find_vertical_spreads(self):
        """Identify vertical spreads from the positions"""
        # Group options by symbol and expiry
        option_groups = {}
        for key, data in self.positions.items():
            symbol, sec_type, strike, right, expiry = key
            if sec_type == "OPT":
                group_key = (symbol, expiry, right)  # Group by symbol, expiry, and call/put
                if group_key not in option_groups:
                    option_groups[group_key] = []
                option_groups[group_key].append((strike, data))
        
        # Find vertical spreads in each group
        for group_key, options in option_groups.items():
            symbol, expiry, right = group_key
            
            # Need at least 2 options of same type (calls or puts) to form a spread
            if len(options) >= 2:
                options.sort()  # Sort by strike price
                
                # Check each pair of adjacent strikes
                for i in range(len(options) - 1):
                    low_strike, low_data = options[i]
                    high_strike, high_data = options[i + 1]
                    
                    low_pos = low_data['position']
                    high_pos = high_data['position']
                    
                    # If positions have opposite signs, it might be a vertical spread
                    # But for this script, we only want positions > 0
                    if low_pos > 0 and high_pos > 0:
                        spread_type = "Bull" if (right == "C" and low_pos > 0) or (right == "P" and low_pos < 0) else "Bear"
                        spread_info = {
                            'symbol': symbol,
                            'expiry': expiry,
                            'right': right,  # "C" for call, "P" for put
                            'low_strike': low_strike,
                            'high_strike': high_strike,
                            'low_position': low_pos,
                            'high_position': high_pos,
                            'type': spread_type,
                            'low_contract': low_data['contract'],
                            'high_contract': high_data['contract']
                        }
                        self.vertical_spreads.append(spread_info)
                        
                        # Request market data for the spread components if not already requested
                        low_key = (symbol, "OPT", low_strike, right, expiry)
                        high_key = (symbol, "OPT", high_strike, right, expiry)
                        
                        if low_key not in self.options_data:
                            req_id = self.nextOrderId()
                            self.req_id_to_option[req_id] = low_key
                            self.reqMktData(req_id, low_data['contract'], "", False, False, [])
                        
                        if high_key not in self.options_data:
                            req_id = self.nextOrderId()
                            self.req_id_to_option[req_id] = high_key
                            self.reqMktData(req_id, high_data['contract'], "", False, False, [])

    def get_all_positions_dataframe(self):
        """Return a pandas DataFrame with all position information"""
        if not self.positions:
            print("No positions found.")
            return pd.DataFrame()
        
        # Create a list to store data for the DataFrame
        data = []
        
        for key, position_data in self.positions.items():
            symbol, sec_type, strike, right, expiry = key
            contract = position_data['contract']
            position_size = position_data['position']
            avg_cost = position_data['avgCost']
            account = position_data['account']
            
            # Get current market price if available
            current_price = None
            market_value = None
            unrealized_pnl = None
            
            if key in self.options_data and 'price' in self.options_data[key]:
                current_price = self.options_data[key]['price']
                
                # Calculate market value
                if sec_type == "OPT":
                    # Options are for 100 shares
                    market_value = current_price * position_size * 100
                    cost_basis = avg_cost * position_size * 100
                else:
                    market_value = current_price * position_size
                    cost_basis = avg_cost * position_size
                
                # Calculate unrealized P&L
                unrealized_pnl = market_value - cost_basis
            
            # Prepare row data based on security type
            row_data = {
                'Account': account,
                'Symbol': symbol,
                'SecType': sec_type,
                'Position': position_size,
                'AvgCost': avg_cost,
                'CurrentPrice': current_price,
                'MarketValue': market_value,
                'UnrealizedPnL': unrealized_pnl
            }
            
            # Add option-specific fields if it's an option
            if sec_type == "OPT":
                row_data.update({
                    'Strike': strike,
                    'Right': right,
                    'Expiry': expiry
                })
                
                # Create a description field for options
                desc = f"{symbol} {right} {strike} {expiry}"
                row_data['Description'] = desc
            else:
                # For non-options, use symbol as description
                row_data['Description'] = symbol
                
                # Add placeholders for option-specific fields
                row_data.update({
                    'Strike': None,
                    'Right': None,
                    'Expiry': None
                })
            
            data.append(row_data)
        
        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Format monetary columns
        for col in ['MarketValue', 'UnrealizedPnL']:
            if col in df.columns and not df[col].isna().all():
                df[col] = df[col].apply(lambda x: f"${x:.2f}" if pd.notnull(x) else "N/A")
        
        # Sort by symbol and then security type
        if not df.empty:
            df = df.sort_values(['Symbol', 'SecType', 'Strike', 'Expiry'])
        
        return df
        
    def get_vertical_spreads_dataframe(self):
        """Return a pandas DataFrame with vertical spread information"""
        if not self.vertical_spreads:
            print("No vertical spreads with position > 0 found.")
            return pd.DataFrame()
        
        # Create a list to store data for the DataFrame
        data = []
        
        for spread in self.vertical_spreads:
            symbol = spread['symbol']
            expiry = spread['expiry']
            right = spread['right']
            low_strike = spread['low_strike']
            high_strike = spread['high_strike']
            low_pos = spread['low_position']
            high_pos = spread['high_position']
            spread_type = f"{spread['type']} {right}"
            
            # Calculate market value if we have price data
            market_value = None
            low_price = None
            high_price = None
            total_value = None
            
            low_key = (symbol, "OPT", low_strike, right, expiry)
            high_key = (symbol, "OPT", high_strike, right, expiry)
            
            if low_key in self.options_data and high_key in self.options_data:
                if 'price' in self.options_data[low_key] and 'price' in self.options_data[high_key]:
                    low_price = self.options_data[low_key]['price']
                    high_price = self.options_data[high_key]['price']
                    
                    # Market value calculation
                    low_value = low_price * low_pos * 100  # Each option contract is for 100 shares
                    high_value = high_price * high_pos * 100
                    total_value = low_value + high_value
            
            # Add row to our data
            data.append({
                'Symbol': symbol,
                'Expiry': expiry,
                'Type': spread_type,
                'Low Strike': low_strike,
                'High Strike': high_strike,
                'Low Position': low_pos,
                'High Position': high_pos,
                'Low Price': low_price,
                'High Price': high_price,
                'Market Value': total_value
            })
        
        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Format the Market Value column for better readability if it contains values
        if 'Market Value' in df.columns and not df['Market Value'].isna().all():
            df['Market Value'] = df['Market Value'].apply(lambda x: f"${x:.2f}" if pd.notnull(x) else "N/A")
        
        return df


# Create and connect the app
app = IBKRApp()

# Adjust these parameters as needed
# For paper trading, default port is 4002
# For live trading, default port is 4001
host = '127.0.0.1'
port = 4002  # Change to 4001 for live trading
client_id = random.randint(1000, 9999) 

print(f"Connecting to IB Gateway at {host}:{port}...")
print("Attempting to connect... Make sure IB Gateway is running and API connections are enabled.")
print("If using a paper account, port should be 4002.")
print("If using a live account, port should be 4001.")

app.connectTWS(host, port, client_id)

# Check connection status
print(f"Connection status: {'Connected' if app.isConnectedAndReady() else 'Not connected'}")

if app.isConnectedAndReady():
    print("Connected to IBKR. Requesting positions...")
    
    # Request positions explicitly
    app.reqPositions()
    
    # Wait for positions with timeout
    timeout = 20  # seconds
    start_time = time.time()
    
    # Wait loop with progress indicator
    print("Waiting for position data...")
    while not app.position_end_received and time.time() - start_time < timeout:
        time.sleep(1)
        print(".", end="", flush=True)
    print()
    
    # Whether or not we got the position_end event, try to process what we have
    print(f"Position data received: {'Yes' if app.position_end_received else 'No, proceeding anyway'}")
    print(f"Number of positions found: {len(app.positions)}")
    
    # If we have any positions, or if positionEnd was called, proceed
    if len(app.positions) > 0 or app.position_end_received:
        # Wait for market data
        print("Waiting for market data...")
        time.sleep(5)  # Extended wait for market data
        
        # Get all positions as a DataFrame
        positions_df = app.get_all_positions_dataframe()
        
        # Display all positions
        print("\n=== All Open Positions ===")
        if not positions_df.empty:
            display(positions_df)
        else:
            print("No open positions found.")
        
        # Also get and display vertical spreads if there are any
        if app.position_end_received:  # Only try to get spreads if we properly received all positions
            app.find_vertical_spreads()  # Make sure vertical spreads are identified
            spreads_df = app.get_vertical_spreads_dataframe()
            
            if not spreads_df.empty:
                print("\n=== Vertical Spread Positions ===")
                display(spreads_df)
        
        # Disconnect
        app.disconnect()
        print("\nDisconnected from IB Gateway")
        
        # Return the DataFrame of all positions for further analysis
        positions_df
    else:
        print("No positions found or position data incomplete.")
        app.disconnect()
        print("\nDisconnected from IB Gateway")
else:
    print("Failed to connect to IB Gateway.")
    print("Please check that:")
    print("1. IB Gateway is running")
    print("2. API connections are enabled in IB Gateway settings")
    print("3. You are using the correct port number")
    print("4. Your firewall is not blocking the connection")

In [ ]:
#!/usr/bin/env python3
"""
Enhanced IBKR Position Analyzer

This script connects to your Interactive Brokers account and retrieves comprehensive
information about your open positions, including entry dates and transaction costs.
It uses multiple IBKR API methods to gather the most complete information.

Requirements:
- ibapi package (pip install ibapi)
- pandas (pip install pandas)
- Interactive Brokers IB Gateway running
"""

import time
import threading
import pandas as pd
import random
import datetime
from dateutil import parser
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import TickerId, BarData
from ibapi.order import Order
from ibapi.order_state import OrderState
from ibapi.execution import ExecutionFilter


class IBKRApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        
        # Store positions and market data
        self.positions = {}  # Dictionary to store position data
        self.options_data = {}  # Dictionary to store option market data
        self.vertical_spreads = []  # List to store identified vertical spreads
        self.req_id_to_option = {}  # Maps request IDs to option contracts
        self.next_req_id = 1
        self.position_end_received = False
        self.account_update_end_received = False
        self.connection_problem = False
        
        # For executions (trades)
        self.executions = {}  # Dictionary to store execution data
        self.exec_details_end_received = False
        
        # For order history
        self.order_history = {}  # Dictionary to store order history
        self.order_history_end_received = False
        self.orders_by_perm_id = {}  # Orders indexed by permanent ID
        self.req_id_to_contract = {}  # Maps request IDs to contracts for order status
        
        # For account summary
        self.account_info = {}
        self.account_summary_end_received = False
        
    def nextOrderId(self):
        """Get the next request ID and increment the counter"""
        id = self.next_req_id
        self.next_req_id += 1
        return id
    
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        """Handle error messages from IB Gateway"""
        # Informational messages
        if errorCode in [2104, 2106, 2158]:
            # These are connection confirmations, not errors
            return
        
        # Actual errors
        if errorCode not in [2104, 2106, 2158]:
            print(f"Error {errorCode}: {errorString}")
            if errorCode == 504:
                self.connection_problem = True
    
    def isConnectedAndReady(self):
        """Check if we're connected and no connection problems reported"""
        return self.isConnected() and not self.connection_problem
    
    def connectTWS(self, host='127.0.0.1', port=4002, clientId=1):
        """Connect to IB Gateway"""
        self.connect(host, port, clientId)
        
        # Start a thread to process messages
        thread = threading.Thread(target=self.run)
        thread.daemon = True
        thread.start()
        
        # Allow time for connection to establish
        time.sleep(2)
    
    def position(self, account, contract, position, avgCost):
        """Handle position data"""
        if position != 0:  # Consider all non-zero positions
            print(f"Position received: {contract.symbol}, {position} @ {avgCost}")
            
            # Store basic position info
            # For options, include strike, right, and expiry
            if contract.secType == "OPT":
                key = (contract.symbol, contract.secType, contract.strike, contract.right, contract.lastTradeDateOrContractMonth)
            else:
                # For stocks and other instruments
                key = (contract.symbol, contract.secType, None, None, None)
                
            self.positions[key] = {
                'contract': contract,
                'position': position,
                'avgCost': avgCost,
                'account': account,
                'conId': contract.conId if hasattr(contract, 'conId') else None
            }
            
            # Request market data for this instrument
            req_id = self.nextOrderId()
            self.req_id_to_option[req_id] = key
            self.reqMktData(req_id, contract, "", False, False, [])
    
    def positionEnd(self):
        """Called when all positions have been reported"""
        print("Position end received")
        self.position_end_received = True
        
        # Request order history for each position to get opening date
        self.requestOrderHistory()
    
    def requestExecutions(self):
        """Request execution data for trades"""
        print("Requesting execution data...")
        try:
            # Create a filter that gets all executions
            exec_filter = ExecutionFilter()
            # Leave all filter fields empty to get all executions
            
            # Set time parameter to get a longer history (up to 90 days)
            # Format: YYYYMMDD-HH:MM:SS
            ninety_days_ago = datetime.datetime.now() - datetime.timedelta(days=90)
            exec_filter.time = ninety_days_ago.strftime("%Y%m%d-%H:%M:%S")
            
            self.reqExecutions(self.nextOrderId(), exec_filter)
        except Exception as e:
            print(f"Error requesting executions: {e}")
            # Set flag to continue anyway
            self.exec_details_end_received = True
    
    def execDetails(self, reqId, contract, execution):
        """Called when execution data is received"""
        print(f"Execution received: {contract.symbol}, {execution.side} {execution.shares} @ {execution.price} on {execution.time}")
        
        # Create a unique key for this execution
        # For options we need to include all contract details
        if contract.secType == "OPT":
            key = (contract.symbol, contract.secType, contract.strike, contract.right, contract.lastTradeDateOrContractMonth)
        else:
            key = (contract.symbol, contract.secType, None, None, None)
            
        # Store execution data
        if key not in self.executions:
            self.executions[key] = []
            
        # Check what attributes are available in the execution object
        # Some versions of IBKR API might not include commission
        execution_data = {
            'execId': execution.execId,
            'time': execution.time,  # Format: YYYYMMDD-HH:MM:SS
            'side': execution.side,  # BOT or SLD
            'shares': execution.shares,
            'price': execution.price,
            'contract': contract,
            'orderRef': execution.orderRef if hasattr(execution, 'orderRef') else '',
            'permId': execution.permId if hasattr(execution, 'permId') else None
        }
        
        # Safely check for commission attribute
        if hasattr(execution, 'commission'):
            execution_data['commission'] = execution.commission
        else:
            execution_data['commission'] = 0.0  # Default if not available
            
        self.executions[key].append(execution_data)
        
    def execDetailsEnd(self, reqId):
        """Called when all execution data has been received"""
        print(f"Execution data received. Found {sum(len(execs) for execs in self.executions.values())} executions.")
        self.exec_details_end_received = True
    
    def requestOrderHistory(self):
        """Request order history to get opening dates for positions"""
        print("Requesting order history...")
        try:
            # Request order history for the last 180 days
            today = datetime.datetime.now().strftime("%Y%m%d-%H:%M:%S")
            six_months_ago = (datetime.datetime.now() - datetime.timedelta(days=180)).strftime("%Y%m%d-%H:%M:%S")
            
            # Request completed orders
            self.reqCompletedOrders(False)  # False = no all-or-none orders
            
            # For each position, also request detailed contract info
            for key, pos_data in self.positions.items():
                contract = pos_data['contract']
                req_id = self.nextOrderId()
                self.req_id_to_contract[req_id] = key
                self.reqContractDetails(req_id, contract)
        except Exception as e:
            print(f"Error requesting order history: {e}")
            self.order_history_end_received = True
    
    def completedOrder(self, contract, order, orderState):
        """Called with completed order information"""
        # Create a key for the contract
        if contract.secType == "OPT":
            key = (contract.symbol, contract.secType, contract.strike, contract.right, contract.lastTradeDateOrContractMonth)
        else:
            key = (contract.symbol, contract.secType, None, None, None)
        
        # Create order record
        order_data = {
            'contract': contract,
            'permId': order.permId,
            'action': order.action,  # BUY or SELL
            'totalQuantity': order.totalQuantity,
            'orderType': order.orderType,
            'lmtPrice': order.lmtPrice,
            'auxPrice': order.auxPrice,
            'status': orderState.status,
            'filled': order.filledQuantity,
            'remaining': order.totalQuantity - order.filledQuantity,
            'avgFillPrice': order.avgFillPrice,
            'lastFillPrice': order.lastFillPrice,
            'whyHeld': orderState.whyHeld,
            'time': order.activeStartTime or "",  # When the order became active
            'commission': orderState.commission if hasattr(orderState, 'commission') and orderState.commission is not None else 0.0
        }
        
        # Store by contract key
        if key not in self.order_history:
            self.order_history[key] = []
        self.order_history[key].append(order_data)
        
        # Also store by permanent ID for easy lookup
        self.orders_by_perm_id[order.permId] = order_data
        
    def completedOrdersEnd(self):
        """Called when all completed orders have been received"""
        print(f"Order history received. Found {sum(len(orders) for orders in self.order_history.values())} orders.")
        self.order_history_end_received = True
        
    def contractDetails(self, reqId, contractDetails):
        """Called when contract details are received"""
        if reqId in self.req_id_to_contract:
            key = self.req_id_to_contract[reqId]
            if key in self.positions:
                # Update the position with additional contract info
                self.positions[key]['contractDetails'] = contractDetails
                
    def contractDetailsEnd(self, reqId):
        """Called when all contract details have been received"""
        if reqId in self.req_id_to_contract:
            print(f"Contract details received for {self.req_id_to_contract[reqId]}")
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Handle price updates for options"""
        if reqId in self.req_id_to_option and tickType == 4:  # 4 is last price
            option_key = self.req_id_to_option[reqId]
            if option_key not in self.options_data:
                self.options_data[option_key] = {}
            self.options_data[option_key]['price'] = price

    def get_all_positions_dataframe(self):
        """Return a pandas DataFrame with all position information including opening dates and costs"""
        if not self.positions:
            print("No positions found.")
            return pd.DataFrame()
        
        # Create a list to store data for the DataFrame
        data = []
        
        for key, position_data in self.positions.items():
            symbol, sec_type, strike, right, expiry = key
            contract = position_data['contract']
            position_size = position_data['position']
            avg_cost = position_data['avgCost']
            account = position_data['account']
            
            # Get current market price if available
            current_price = None
            market_value = None
            unrealized_pnl = None
            
            if key in self.options_data and 'price' in self.options_data[key]:
                current_price = self.options_data[key]['price']
                
                # Calculate market value
                if sec_type == "OPT":
                    # Options are for 100 shares
                    market_value = current_price * position_size * 100
                    cost_basis = avg_cost * position_size * 100
                else:
                    market_value = current_price * position_size
                    cost_basis = avg_cost * position_size
                
                # Calculate unrealized P&L
                unrealized_pnl = market_value - cost_basis
            
            # Get opening date and transaction cost
            open_date = "N/A"
            transaction_cost = "N/A"
            commission = 0.0
            
            # Try to get from order history first (most reliable)
            if key in self.order_history:
                # Find matching orders (BUY for long positions, SELL for short positions)
                target_action = "BUY" if position_size > 0 else "SELL"
                matching_orders = [o for o in self.order_history[key] 
                                  if o["action"] == target_action and o["status"] == "Filled"]
                
                if matching_orders:
                    # Sort by time to get the earliest order
                    matching_orders.sort(key=lambda x: x["time"] if x["time"] else "99999999-99:99:99")
                    first_order = matching_orders[0]
                    
                    # Format the timestamp
                    if first_order["time"]:
                        try:
                            # Parse the timestamp
                            dt = parser.parse(first_order["time"])
                            open_date = dt.strftime("%Y-%m-%d %H:%M:%S")
                        except:
                            open_date = first_order["time"]
                    
                    # Get commission
                    commission = first_order.get("commission", 0.0)
                    
                    # Calculate transaction cost
                    if sec_type == "OPT":
                        # Options are for 100 shares
                        price = first_order["avgFillPrice"] or first_order["lmtPrice"]
                        transaction_cost = (price * abs(first_order["totalQuantity"]) * 100) + commission
                    else:
                        price = first_order["avgFillPrice"] or first_order["lmtPrice"]
                        transaction_cost = (price * abs(first_order["totalQuantity"])) + commission
            
            # If not found in order history, try executions
            if open_date == "N/A" and key in self.executions and self.executions[key]:
                target_side = "BOT" if position_size > 0 else "SLD"
                matching_execs = [e for e in self.executions[key] if e["side"] == target_side]
                
                if matching_execs:
                    # Sort by time to get the earliest execution
                    matching_execs.sort(key=lambda x: x["time"])
                    first_exec = matching_execs[0]
                    
                    # Format the timestamp
                    try:
                        # Parse the timestamp (YYYYMMDD-HH:MM:SS)
                        dt = parser.parse(first_exec["time"])
                        open_date = dt.strftime("%Y-%m-%d %H:%M:%S")
                    except:
                        open_date = first_exec["time"]
                    
                    # Extract commission safely
                    commission = first_exec.get("commission", 0.0)
                        
                    # Calculate total cost including commission
                    if sec_type == "OPT":
                        # Options are for 100 shares
                        transaction_cost = (first_exec["price"] * abs(first_exec["shares"]) * 100) + commission
                    else:
                        transaction_cost = (first_exec["price"] * abs(first_exec["shares"])) + commission
            
            # Prepare row data based on security type
            row_data = {
                'Account': account,
                'Symbol': symbol,
                'SecType': sec_type,
                'Position': position_size,
                'AvgCost': avg_cost,
                'CurrentPrice': current_price,
                'MarketValue': market_value,
                'UnrealizedPnL': unrealized_pnl,
                'OpenDate': open_date,
                'TransactionCost': transaction_cost,
                'Commission': commission
            }
            
            # Add option-specific fields if it's an option
            if sec_type == "OPT":
                row_data.update({
                    'Strike': strike,
                    'Right': right,
                    'Expiry': expiry
                })
                
                # Create a description field for options
                desc = f"{symbol} {right} {strike} {expiry}"
                row_data['Description'] = desc
            else:
                # For non-options, use symbol as description
                row_data['Description'] = symbol
                
                # Add placeholders for option-specific fields
                row_data.update({
                    'Strike': None,
                    'Right': None,
                    'Expiry': None
                })
            
            data.append(row_data)
        
        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Format monetary columns
        for col in ['MarketValue', 'UnrealizedPnL', 'TransactionCost']:
            if col in df.columns and not df[col].isna().all():
                df[col] = df[col].apply(lambda x: f"${x:.2f}" if pd.notnull(x) and x != "N/A" else x)
        
        # Sort by symbol and then security type
        if not df.empty:
            df = df.sort_values(['Symbol', 'SecType', 'Strike', 'Expiry'])
        
        return df


# Create and connect the app
app = IBKRApp()

# Adjust these parameters as needed
# For paper trading, default port is 4002
# For live trading, default port is 4001
host = '127.0.0.1'
port = 4002  # Change to 4001 for live trading
client_id = random.randint(1000, 9999)  # Generate a random client ID between 1000 and 9999

print(f"Connecting to IB Gateway at {host}:{port} with client ID: {client_id}")
print("Attempting to connect... Make sure IB Gateway is running and API connections are enabled.")
print("If using a paper account, port should be 4002.")
print("If using a live account, port should be 4001.")

app.connectTWS(host, port, client_id)

# Check connection status
print(f"Connection status: {'Connected' if app.isConnectedAndReady() else 'Not connected'}")

if app.isConnectedAndReady():
    print("Connected to IBKR. Requesting data...")
    
    # Request positions
    app.reqPositions()
    
    # Wait for positions with timeout
    timeout = 30  # seconds
    start_time = time.time()
    
    # Wait loop with progress indicator
    print("Waiting for position data...")
    while not app.position_end_received and time.time() - start_time < timeout:
        time.sleep(1)
        print(".", end="", flush=True)
    print()
    
    # Now that we have positions, request executions and order history
    if len(app.positions) > 0:
        # Request executions to get trade timestamps and costs
        app.requestExecutions()
        
        # Wait for executions and order history with timeout
        timeout = 30  # seconds
        start_time = time.time()
        
        # Wait loop with progress indicator
        print("Waiting for execution and order history data...")
        while (not app.exec_details_end_received or not app.order_history_end_received) and time.time() - start_time < timeout:
            time.sleep(1)
            print(".", end="", flush=True)
        print()
    
    # Report on what data we received
    print(f"Position data received: {'Yes' if app.position_end_received else 'No, proceeding anyway'}")
    print(f"Execution data received: {'Yes' if app.exec_details_end_received else 'No, proceeding anyway'}")
    print(f"Order history received: {'Yes' if app.order_history_end_received else 'No, proceeding anyway'}")
    print(f"Number of positions found: {len(app.positions)}")
    
    # Check if we have executions properly
    num_executions = 0
    for key, execs in app.executions.items():
        if isinstance(execs, list):
            num_executions += len(execs)
    print(f"Number of executions found: {num_executions}")
    
    # Check if we have order history properly
    num_orders = 0
    for key, orders in app.order_history.items():
        if isinstance(orders, list):
            num_orders += len(orders)
    print(f"Number of orders found: {num_orders}")
    
    # If we have any positions, proceed
    if len(app.positions) > 0:
        # Wait for market data
        print("Waiting for market data...")
        time.sleep(5)  # Extended wait for market data
        
        # Get all positions as a DataFrame
        positions_df = app.get_all_positions_dataframe()
        
        # Display all positions
        print("\n=== All Open Positions ===")
        if not positions_df.empty:
            display(positions_df)
        else:
            print("No open positions found.")
        
        # Disconnect
        app.disconnect()
        print("\nDisconnected from IB Gateway")
        
        # Return the DataFrame of all positions for further analysis
        positions_df
    else:
        print("No positions found or position data incomplete.")
        app.disconnect()
        print("\nDisconnected from IB Gateway")
else:
    print("Failed to connect to IB Gateway.")
    print("Please check that:")
    print("1. IB Gateway is running")
    print("2. API connections are enabled in IB Gateway settings")
    print("3. You are using the correct port number")
    print("4. Your firewall is not blocking the connection")

In [12]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.execution import ExecutionFilter
import threading
import time
import pandas as pd

class IBApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.trades = []  # To store execution details

    def execDetails(self, reqId, contract, execution):
        print(f"Trade - Symbol: {contract.symbol}, Side: {execution.side}, Shares: {execution.shares}, Price: {execution.price}")
        # Save to list
        self.trades.append({
            "symbol": contract.symbol,
            "side": execution.side,
            "shares": execution.shares,
            "price": execution.price,
            "time": execution.time
        })

    def execDetailsEnd(self, reqId):
        print(f"Finished receiving executions for request {reqId}.")

    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson=""):
        # Only show important errors
        if errorCode not in [2104, 2106, 2158]:
            print(f"Error {errorCode}: {errorString}")

def run_loop(app):
    app.run()

# --- Main run starts here ---

app = IBApp()
app.connect("127.0.0.1", 7497, clientId=1)

# Start the run loop in a background thread
api_thread = threading.Thread(target=run_loop, args=(app,), daemon=True)
api_thread.start()

# Wait a bit to ensure connection is ready
time.sleep(2)

# Create a blank ExecutionFilter to pull all trades
exec_filter = ExecutionFilter()

# Request trade history
app.reqExecutions(1, exec_filter)

# Allow time to receive data
time.sleep(5)

# Disconnect cleanly
app.disconnect()

# --- Save trades to CSV ---
if app.trades:
    df = pd.DataFrame(app.trades)
    df.to_csv('trade_history.csv', index=False)
    print("✅ Trades saved to 'trade_history.csv'")
else:
    print("⚠️ No trades found.")


Error 1745647300828: 504
⚠️ No trades found.
